# Spark RDD Fundamentals



### Import libraries

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext
import numpy as np

## Initialize a `SparkContext` (the main abstraction to the cluster)
**Note the '4' in the argument. It denotes 4 cores to be used for this SparkContext object.**

In [3]:
sc=SparkContext(master="local[4]")

In [4]:
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


### Generate a list of random integeres

In [5]:
lst=np.random.randint(0,10,20)

In [6]:
print(lst)

[6 4 6 1 2 0 8 3 2 3 9 9 3 5 0 7 8 7 2 6]


### Parallelize the list - this is the main operation toward distributed computing

In [8]:
A=sc.parallelize(lst)

### `A` is a pyspark RDD object, we cannot access the elements directly

In [9]:
type(A)

pyspark.rdd.RDD

In [10]:
A

ParallelCollectionRDD[1] at parallelize at PythonRDD.scala:195

### Opposite to parallelization - `collect` brings all the distributed elements and returns them to the head node. <br><br>Note - this is a slow process, do not use it often. 

In [11]:
A.collect()

[6, 4, 6, 1, 2, 0, 8, 3, 2, 3, 9, 9, 3, 5, 0, 7, 8, 7, 2, 6]

### How were the partitions created? Use `glom` method

In [12]:
A.glom().collect()

[[6, 4, 6, 1, 2], [0, 8, 3, 2, 3], [9, 9, 3, 5, 0], [7, 8, 7, 2, 6]]

### Now stop the SC and reinitialize it with 2 cores and see what happens when you repeat the process!

In [13]:
sc.stop()

In [14]:
sc=SparkContext(master="local[2]")

In [15]:
A = sc.parallelize(lst)

In [16]:
A.glom().collect()

[[6, 4, 6, 1, 2, 0, 8, 3, 2, 3], [9, 9, 3, 5, 0, 7, 8, 7, 2, 6]]

**The RDD is now distributed over two chunks, not four!** 

So, let's redo the process with 4 cores again.

In [17]:
sc.stop()

In [18]:
sc = SparkContext(master="local[4]")

In [19]:
A = sc.parallelize(lst)

In [20]:
#A = sc.parallelize(range(1000000),numSlices=8)

## Basic operations
### `Count` the elements

In [21]:
A.count()

20

### The first element (`first`) and the first few elements (`take`)

In [22]:
A.first()

6

In [23]:
A.take(4)

[6, 4, 6, 1]

### Removing duplicates: Get another RDD with only the `distinct` elements

The method `RDD.distinct()` Returns a new dataset that contains the distinct elements of the source dataset.

**NOTE**: This operation requires a **shuffle** in order to detect duplication across partitions. **So, it is a slow operation.**

In [24]:
A_distinct=A.distinct()

In [26]:
A_distinct.collect()

[4, 0, 8, 1, 9, 5, 6, 2, 3, 7]

### To sum all the elements use `reduce` method

In [27]:
A.reduce(lambda x,y:x+y)

91

### Or direct `sum` method

In [28]:
A.sum()

91

### Or using the `fold` method, which aggregates the elements of each partition, and then the results for all the partitions

In [29]:
A.fold(1,lambda x,y:x+y)

96

### Finding maximum element by `reduce`

In [30]:
A.reduce(lambda x,y: x if x > y else y)

9

### Finding longest word using `reduce`

In [31]:
words = 'Spark is Supercalifragilisticexpialidocious and next best thing after sliced bread'.split(' ')
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)

'Supercalifragilisticexpialidocious'

## Functions/filtering over RDD
### Use `filter` to return a new RDD with elements satisfying a given predicate (lambda expression)

In [32]:
# Return RDD with elements divisible by 3
A.filter(lambda x:x%3==0).collect()

[6, 6, 0, 3, 3, 9, 9, 3, 0, 6]

### Lambda functions are short and sweet but we can write regular Python functions to use with `reduce`

In [33]:
def largerThan(x,y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x)> len(y):
        return x
    elif len(y) > len(x):
        return y
    else:
        if x < y: return x
        else: return y

In [34]:
wordRDD.reduce(largerThan)

'Supercalifragilisticexpialidocious'

## Sampling an RDD
* RDDs are often very large.
* **Aggregates, such as averages, can be approximated efficiently by using a sample.** This comes handy often for operation with extremely large datasets where a sample can tell a lot about the pattern and descriptive statistics of the data.
* Sampling is done in parallel and requires limited computation.

The method `RDD.sample(withReplacement,p)` generates a sample of the elements of the RDD. where
- `withReplacement` is a boolean flag indicating whether or not a an element in the RDD can be sampled more than once.
- `p` is the probability of accepting each element into the sample. Note that as the sampling is performed independently in each partition, the number of elements in the sample changes from sample to sample.

In [35]:
# get a sample whose expected size is m
# Note that the size of the sample is different in different runs
m=5
n=20
print('sample1=',A.sample(False,m/n).collect()) 
print('sample2=',A.sample(False,m/n).collect())

sample1= [1, 2, 3, 2, 9, 0, 7, 6]
sample2= [6, 8, 9, 3]


## Basic statistics

In [36]:
print("Maximum: ",A.max())
print("Minimum: ",A.min())
print("Mean (average): ",A.mean())
print("Standard deviation: ",A.stdev())

Maximum:  9
Minimum:  0
Mean (average):  4.55
Standard deviation:  2.8543825952384165


In [37]:
A.stats()

(count: 20, mean: 4.55, stdev: 2.8543825952384165, max: 9.0, min: 0.0)

## Mapping
### `map` operation with _lambda_ function

In [43]:
BNewRDD=A.map(lambda x:x/2)

In [44]:
BNewRDD.collect()

[3.0,
 2.0,
 3.0,
 0.5,
 1.0,
 0.0,
 4.0,
 1.5,
 1.0,
 1.5,
 4.5,
 4.5,
 1.5,
 2.5,
 0.0,
 3.5,
 4.0,
 3.5,
 1.0,
 3.0]

### `map` operation with regular Python function

In [45]:
def square_if_odd(x):
    if x%2==1:
        return x*x
    else:
        return x

In [46]:
A.map(square_if_odd).collect()

[6, 4, 6, 1, 2, 0, 8, 9, 2, 9, 81, 81, 9, 25, 0, 49, 8, 49, 2, 6]

### `flatmap` method returns a new RDD by first applying a function to all elements of this RDD, and then flattening the results

In [47]:
A.flatMap(lambda x:(x,x*x)).collect()

[6,
 36,
 4,
 16,
 6,
 36,
 1,
 1,
 2,
 4,
 0,
 0,
 8,
 64,
 3,
 9,
 2,
 4,
 3,
 9,
 9,
 81,
 9,
 81,
 3,
 9,
 5,
 25,
 0,
 0,
 7,
 49,
 8,
 64,
 7,
 49,
 2,
 4,
 6,
 36]

## Grouping and binning
### `groupby` returns a RDD of grouped elements (iterable) as per a given group operation (function)

In [48]:
result=A.groupBy(lambda x:x%2).collect()
sorted([(x, sorted(y)) for (x, y) in result])

[(0, [0, 0, 2, 2, 2, 4, 6, 6, 6, 8, 8]), (1, [1, 3, 3, 3, 5, 7, 7, 9, 9])]

### `histogram` method takes a list of bins/buckets and returns a tuple with result of the histogram (binning) 

In [49]:
B.histogram([x for x in range(0,100,10)])

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [9, 1, 1, 3, 2, 0, 2, 0, 2])

## Set operations
### Create smaller RDDs to demonstrate joint operations

In [50]:
lst1=np.random.randint(0,10,3)
C=sc.parallelize(lst1)
lst2=np.random.randint(10,20,3)
D=sc.parallelize(lst2)
print("C:",C.collect())
print("D:",D.collect())

C: [2, 8, 3]
D: [18, 14, 18]


### `C+D` gives the union (like set union), not the element wise sum

In [51]:
(C+D).collect()

[2, 8, 3, 18, 14, 18]

### `cartesian` gives the pairwise product (as tuples) 

In [52]:
C.cartesian(D).collect()

[(2, 18),
 (2, 14),
 (2, 18),
 (8, 18),
 (8, 14),
 (8, 18),
 (3, 18),
 (3, 14),
 (3, 18)]

### `intersection` and `subtract `methods return a RDD of the set intersection and subtraction (difference)

In [53]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

[1, 2, 3]

In [54]:
rdd1.subtract(rdd2).collect()

[10, 4, 5]

### Stop the `SparkContext` at the end

In [55]:
sc.stop()